## Demo Skript to determine single pictures of bunnies from group photos

In [1]:
import sys
import os

# Add the project root to sys.path so 'config' can be imported
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import torch
from torchvision import transforms
from PIL import Image, ImageDraw
import os
import pytorch_lightning as pl

from config.load_configuration import load_configuration
config = load_configuration()

pl.seed_everything(config['seed'])
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   # disable oneDNN optimizations for reproducibility

Seed set to 42


PC Name: DESKTOP-MIKA
Loaded configuration from ../config/config_mika.yaml


In [2]:
import torchvision
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

f:\Users\Mika\Anaconda\envs\VDKI-Projekt\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
f:\Users\Mika\Anaconda\envs\VDKI-Projekt\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
model.eval()

# COCO class index for rabbit is not present, so let's use 'rabbit' from a custom model or fallback to 'dog' or 'cat' for demonstration.
# For real rabbit detection, you would need a model trained to detect rabbits specifically.

# Helper function to get bounding boxes for rabbits
def get_rabbit_boxes(image, threshold=0.5):                                                         #TODO: adjust threshold as necessary
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    img_tensor = transform(image)
    with torch.no_grad():
        prediction = model([img_tensor])[0]
    boxes = []
    labels = prediction['labels'].tolist()
    scores = prediction['scores'].tolist()
    # print(labels, scores)                                                                         #TODO: show labels and scores
    accepted_labels = [88, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]  # COCO classes for animals      #TODO: add labels as necessary
    # COCO: 17=cat, 18=dog, 19=horse, 20=sheep, 21=cow, 22=elephant, 23=bear, 24=zebra, 25=giraffe
    for box, label, score in zip(prediction['boxes'], labels, scores):
        if label in accepted_labels and score > threshold:
            boxes.append(box.int().tolist())
    return boxes

def save_rabbit_crops(image_path, border=10):
    image = Image.open(image_path).convert("RGB")
    import matplotlib.pyplot as plt

    # plt.imshow(image) # Uncomment to visualize the image                                          #TODO: Kernel crashes
    # plt.axis('off')
    # plt.show()

    boxes = get_rabbit_boxes(image)
    print(f"Found {len(boxes)} rabbit-like objects in {image_path}.")
    base, ext = os.path.splitext(image_path)
    for idx, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        # Add border, ensuring we don't go out of image bounds
        x1b = max(0, x1 - border)
        y1b = max(0, y1 - border)
        x2b = min(image.width, x2 + border)
        y2b = min(image.height, y2 + border)
        crop = image.crop((x1b, y1b, x2b, y2b))
        crop.save(f"{base}_{idx+1}{ext}")

# Example usage:
save_rabbit_crops('IMG_0140.jpg', border=20)


Found 3 rabbit-like objects in IMG_0140.jpg.
